In [1]:
import pandas as pd
import numpy as np
import glob

### 0. Importando csvs e juntando em uma tabela principal

In [2]:
band = pd.read_csv('../debate_segments/2020/processed/band_1o_turno.csv', index_col=0)
band['debate'] = 'BAND 1T'
cultura = pd.read_csv('../debate_segments/2020/processed/cultura_1o_turno.csv', index_col=0)
cultura['debate'] = 'CULTURA 1T'
folha = pd.read_csv('../debate_segments/2020/processed/folha_1o_turno.csv', index_col=0)
folha['debate'] = 'FOLHA 1T'
cnn = pd.read_csv('../debate_segments/2020/processed/cnn_2o_turno.csv', index_col=0)
cnn['debate'] = 'CNN 2T'
band_2 = pd.read_csv('../debate_segments/2020/processed/band_2o_turno.csv', index_col=0)
band_2['debate'] = 'BAND 2T'
rodaviva = pd.read_csv('../debate_segments/2020/processed/rodaviva_2o_turno.csv', index_col=0)
rodaviva['debate'] = 'RODAVIVA 2T'

falas_todas = cultura.append(band, ignore_index=True)
falas_todas = falas_todas.append(folha, ignore_index=True)
falas_todas = falas_todas.append(cnn, ignore_index=True)
falas_todas = falas_todas.append(band_2, ignore_index=True)
falas_todas = falas_todas.append(rodaviva, ignore_index=True)

falas_todas['tempo medio pausa'] = (falas_todas['Duração'] - falas_todas['Tempo de fala']) / falas_todas['Num. pausas']

In [3]:
falas_todas.head()

,fala_id,candidato,tema,Num. sílabas,Num. pausas,Duração,Tempo de fala,Ritmo de fala,Ritmo de articulação,intensity mean,intensity quant,pitch mean,pitch var fala,pitch var sil,debate,ordem,tipo,tempo medio pausa
0,covas_9_imposto.wav,Covas,imposto,219,6,59.297460,57.025460,3.693244,3.840390,69.700465,72.248331,121.662181,59.483780,8.929488,CULTURA 1T,NaN,NaN,0.378667
1,covas_10_corrupcao.wav,Covas,corrupcao,123,1,28.884036,28.468036,4.258408,4.320635,69.731234,72.429363,124.021365,50.715439,7.552412,CULTURA 1T,NaN,NaN,0.416000
2,boulos_8_democracia.wav,Boulos,democracia,149,5,29.758866,27.639433,5.006911,5.390849,69.584417,72.472810,121.517333,45.404642,9.389888,CULTURA 1T,NaN,NaN,0.423887
3,boulos_9_servico-publico.wav,Boulos,servico publico,123,6,29.047596,25.843798,4.234430,4.759362,69.607412,72.525275,123.985363,50.402956,16.277176,CULTURA 1T,NaN,NaN,0.533966
4,boulos_3_covid.wav,Boulos,covid,113,10,28.223220,22.823610,4.003795,4.951013,70.029027,72.766547,132.099895,43.624596,11.669913,CULTURA 1T,NaN,NaN,0.539961


### 1. Gerando visao_geral.csv 

In [4]:
boulos_df = falas_todas[falas_todas['candidato'] == 'Boulos']
covas_df  = falas_todas[falas_todas['candidato'] == 'Covas']

boulos = {}
boulos['candidato'] = 'Boulos'
boulos['ritmo de articulação (média)'] = boulos_df['Ritmo de articulação'].mean()
boulos['ritmo de articulação (desvio padrão)'] = np.sqrt(boulos_df['Ritmo de articulação'].var())
boulos['intensidade média (média)'] = boulos_df['intensity mean'].mean()
boulos['intensidade média (desvio padrão)'] = np.sqrt(boulos_df['intensity mean'].var())
boulos['tom médio (média)'] = boulos_df['pitch mean'].mean()
boulos['tom médio (desvio padrão)'] = np.sqrt(boulos_df['pitch mean'].var())
boulos['var. tom (média)'] = boulos_df['pitch var fala'].mean()
boulos['var. tom (desvio padrão)'] = np.sqrt(boulos_df['pitch var fala'].var())

covas = {}
covas['candidato'] = 'Covas'
covas['ritmo de articulação (média)'] = covas_df['Ritmo de articulação'].mean()
covas['ritmo de articulação (desvio padrão)'] = np.sqrt(covas_df['Ritmo de articulação'].var())
covas['intensidade média (média)'] = covas_df['intensity mean'].mean()
covas['intensidade média (desvio padrão)'] = np.sqrt(covas_df['intensity mean'].var())
covas['tom médio (média)'] = covas_df['pitch mean'].mean()
covas['tom médio (desvio padrão)'] = np.sqrt(covas_df['pitch mean'].var())
covas['var. tom (média)'] = covas_df['pitch var fala'].mean()
covas['var. tom (desvio padrão)'] = np.sqrt(covas_df['pitch var fala'].var())

visao_geral = pd.DataFrame([boulos, covas])

In [5]:
visao_geral.head()

,candidato,ritmo de articulação (média),ritmo de articulação (desvio padrão),intensidade média (média),intensidade média (desvio padrão),tom médio (média),tom médio (desvio padrão),var. tom (média),var. tom (desvio padrão)
0,Boulos,4.990085,0.315277,70.266024,2.045900,135.732745,15.603636,49.036773,23.135822
1,Covas,4.375609,0.334126,70.502362,2.408127,121.149707,10.024298,33.349219,13.668993


In [6]:
# visao_geral.to_csv('./visao_geral.csv')

### 2. Gerando totais_tema.csv

In [7]:
# Importa o csv que já tem o mapeamento tema -> tema geral
temas_df = pd.read_csv('../debate_segments/2020/processed/todos_com_tema.csv', index_col=0)

In [8]:
# Agrupa por temas, soma a duração das falas por tema e ordena em ordem decrescente
temas_boulos = temas_df[temas_df['candidato'] == 'Boulos'].groupby('tema geral').sum()['Duração'].sort_values(ascending=False)
temas_covas = temas_df[temas_df['candidato'] == 'Covas'].groupby('tema geral').sum()['Duração'].sort_values(ascending=False)

In [9]:
temas_boulos_df = pd.DataFrame(temas_boulos)
temas_boulos_df['candidato'] = 'Boulos'
temas_boulos_df.reset_index(level=0, inplace=True)

temas_covas_df = pd.DataFrame(temas_covas)
temas_covas_df['candidato'] = 'Covas'
temas_covas_df.reset_index(level=0, inplace=True)

totais_por_tema = temas_boulos_df.append(temas_covas_df, ignore_index=True)

In [10]:
totais_por_tema.head()

,tema geral,Duração,candidato
0,economia,1140.916213,Boulos
1,covid/saúde,1117.788954,Boulos
2,moradia,736.650585,Boulos
3,terceirização/serv. público,586.542381,Boulos
4,polarização/radicalismo,580.450068,Boulos


In [11]:
# totais_por_tema.to_csv('./totais_tema.csv')

### 3. Gerando features_por_tema.csv

In [12]:
temas_boulos_df = temas_df[temas_df['candidato'] == 'Boulos']
temas_covas_df = temas_df[temas_df['candidato'] == 'Covas']

In [13]:
temas_boulos_df.groupby('tema geral').mean().index

Index(['alianças', 'apresentacao', 'candidatura', 'considerações finais',
       'corrupção', 'covid/saúde', 'cracolandia', 'creches', 'cultura',
       'economia', 'educação', 'equipe', 'etica', 'experiencia', 'fake news',
       'infraestrutura', 'lixo', 'moradia', 'participacao popular',
       'polarização/radicalismo', 'racismo/minorias', 'renda básica',
       'responsabilidade fiscal', 'ricardo nunes', 'russomano', 'seguranca',
       'segurança', 'terceirização/serv. público'],
      dtype='object', name='tema geral')

In [14]:
features_por_tema = []

for tema in list(temas_boulos_df.groupby('tema geral').mean().index):
    newrow = {}
    df = temas_boulos_df[temas_boulos_df['tema geral'] == tema]
    newrow['candidato'] = 'Boulos'
    newrow['tema'] = tema
    newrow['ritmo de fala (média)'] = df['Ritmo de fala'].mean()
    newrow['ritmo de fala (desvio padrão)'] = np.sqrt(df['Ritmo de fala'].var())
    newrow['tom de voz (média)'] = df['pitch mean'].mean()
    newrow['tom de voz (desvio padrão)'] = np.sqrt(df['pitch mean'].var())
    features_por_tema.append(newrow)
    
for tema in list(temas_covas_df.groupby('tema geral').mean().index):
    newrow = {}
    df = temas_covas_df[temas_covas_df['tema geral'] == tema]
    newrow['candidato'] = 'Covas'
    newrow['tema'] = tema
    newrow['ritmo de fala (média)'] = df['Ritmo de fala'].mean()
    newrow['ritmo de fala (desvio padrão)'] = np.sqrt(df['Ritmo de fala'].var())
    newrow['tom de voz (média)'] = df['pitch mean'].mean()
    newrow['tom de voz (desvio padrão)'] = np.sqrt(df['pitch mean'].var())
    features_por_tema.append(newrow)
    
features_df = pd.DataFrame(features_por_tema)

In [15]:
# Obs: temas que são contemplados em apenas uma fala tem desvio padrão == NaN
features_df.head()

,candidato,tema,ritmo de fala (média),ritmo de fala (desvio padrão),tom de voz (média),tom de voz (desvio padrão)
0,Boulos,alianças,4.321066,0.527681,143.926170,14.074423
1,Boulos,apresentacao,3.545380,NaN,121.309189,NaN
2,Boulos,candidatura,3.536351,NaN,102.530535,NaN
3,Boulos,considerações finais,3.727867,0.358903,124.843329,11.206442
4,Boulos,corrupção,4.072099,0.246850,132.356820,5.819723


In [16]:
# features_df.to_csv('./features_por_tema.csv')

### 4. Gerando boulos_2018_2020.csv

In [17]:
# Pegar csvs de cada ano e rotulá-los (os de 2020 já estão carregados, mas para ficar mais claro carrego de novo)

csv_debates_2018 = []
path = '../debate_segments/2018/processed/*.csv'
for file in glob.glob(path):
    csv_debates_2018.append(file)
    
csv_debates_2020 = []
path = '../debate_segments/2020/processed/*.csv'
for file in glob.glob(path):
    csv_debates_2020.append(file)

In [18]:
all_2018 = pd.DataFrame()
for csv in csv_debates_2018:
    df = pd.read_csv(csv, index_col=0)
    all_2018 = all_2018.append(df, ignore_index=True)
    
all_2020 = pd.DataFrame()
for csv in csv_debates_2020:
    df = pd.read_csv(csv, index_col=0)
    all_2020 = all_2020.append(df, ignore_index=True)

In [19]:
all_2020['ano'] = '2020'
all_2018['ano'] = '2018'
boulos_2018 = all_2018[all_2018['candidato'] == 'Boulos']
boulos_2020 = all_2020[all_2020['candidato'] == 'Boulos']

boulos = boulos_2018.append(boulos_2020, ignore_index=True)
boulos['tempo medio pausa'] = (boulos['Duração'] - boulos['Tempo de fala']) / boulos['Num. pausas']

In [20]:
groups_mean = boulos.groupby('ano').mean()
groups_var = np.sqrt(boulos.groupby('ano').var())

In [21]:
b_2018 = {}
b_2018['ano'] = 2018
b_2018['Ritmo de fala (média)'] = groups_mean.loc['2018']['Ritmo de fala']
b_2018['Ritmo de fala (dp)'] = groups_var.loc['2018']['Ritmo de fala']
b_2018['Tempo médio de pausa (média)'] = groups_mean.loc['2018']['tempo medio pausa']
b_2018['Tempo médio de pausa (dp)'] = groups_var.loc['2018']['tempo medio pausa']
b_2018['Tom de voz médio (média)'] = groups_mean.loc['2018']['pitch mean']
b_2018['Tom de voz médio (dp)'] = groups_var.loc['2018']['pitch mean']
b_2018['Intensidade média (média)'] = groups_mean.loc['2018']['intensity mean']
b_2018['Intensidade média (dp)'] = groups_var.loc['2018']['intensity mean']

b_2020 = {}
b_2020['ano'] = 2020
b_2020['Ritmo de fala (média)'] = groups_mean.loc['2020']['Ritmo de fala']
b_2020['Ritmo de fala (dp)'] = groups_var.loc['2020']['Ritmo de fala']
b_2020['Tempo médio de pausa (média)'] = groups_mean.loc['2020']['tempo medio pausa']
b_2020['Tempo médio de pausa (dp)'] = groups_var.loc['2020']['tempo medio pausa']
b_2020['Tom de voz médio (média)'] = groups_mean.loc['2020']['pitch mean']
b_2020['Tom de voz médio (dp)'] = groups_var.loc['2020']['pitch mean']
b_2020['Intensidade média (média)'] = groups_mean.loc['2020']['intensity mean']
b_2020['Intensidade média (dp)'] = groups_var.loc['2020']['intensity mean']

In [22]:
boulos_comp = pd.DataFrame([b_2018, b_2020])
boulos_comp.head()

,ano,Ritmo de fala (média),Ritmo de fala (dp),Tempo médio de pausa (média),Tempo médio de pausa (dp),Tom de voz médio (média),Tom de voz médio (dp),Intensidade média (média),Intensidade média (dp)
0,2018,4.270233,0.304426,0.460080,0.059239,160.611224,11.897198,70.602187,2.046681
1,2020,4.219304,0.406273,0.528323,0.111846,138.856554,21.600102,70.265695,2.068852


In [23]:
# boulos_comp.to_csv('./boulos_2018_2020.csv')